In [1]:
import torch
from torch import nn

In [2]:
# 自定义网络：LeNet-5
class LeNet5(nn.Module):
    # 定义lenet5的网络结构
    def __init__(self):
        """
        初始化方法：
        1、接收超参数
        """
        # 初始化父类
        super(LeNet5, self).__init__()
        # 定义卷积层
        self.conv2d1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
        # 定义亚采样层（池化层）
        self.maxpool2d1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # 定义卷积层
        self.conv2d2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        # 定义亚采样层（池化层）
        self.maxpool2d2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # 三个全连接层
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    # 正向传播
    def forward(self, x):
        x = self.conv2d1(x)
        x = self.maxpool2d1(x)
        x = self.conv2d2(x)
        x = self.maxpool2d2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [3]:
# 打包数据
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import os
from torchvision import transforms

In [4]:
# 自定义Compose转换过程
trans = transforms.Compose(transforms=[
    # 将图像缩放为 32*32 【因为LeNet的要求是32x32的，这里为了全完模拟LeNet，所以对数据做调整，在实际中，可以调整模型而不处理数据】
    # transforms.Resize(size=(32, 32)), 这种是硬拉伸【不推荐】
    # transforms.CenterCrop(size=(32, 32)), 这种是以中心为原点，直接裁剪指定的大小，如果裁大了，会给补偿黑色背景
    transforms.Resize(size=(32, 32)),
    # LeNet要求图像通道是1层，这里也是处理数据，不处理模型。在实际中，可以调整模型而不处理数据
    transforms.Grayscale(),
    transforms.ToTensor(),
    # 这里标准化的时候，图像有3层，就要写三次, transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
    # 因为我们在上面通过 Grayscale 转了灰度，只有一层，所以这里只有一个
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [5]:
class ImageDataset(Dataset):
    """
        自定义数据集
    """
    def __init__(self, root, trans):
        self.root = root
        self.trans = trans
        self.img_files = []
        self.img_labels = []
        self.label2idx = {f"G{idx}": idx for idx in range(10)}
        self.idx2label = {v: k for k, v in self.label2idx.items()}
        self._get_file_info()
    
    def _get_file_info(self):
        for label in os.listdir(self.root):
            # print(label)
            label_root = os.path.join(self.root, label)
            # print(label_root)
            for file in os.listdir(label_root):
                file_path = os.path.join(label_root, file)
                self.img_files.append(file_path)
                self.img_labels.append(label)

    def __len__(self):
        """
            返回数据集中有多少数据
        """
        return len(self.img_files)

    def __getitem__(self, idx):
        """
            读取其中一个样本
        """
        # 图像地址
        img_file = self.img_files[idx]
        # 图像标签
        img_label = self.img_labels[idx]

        # 读取图像
        # 这里除了可以使用 Image 读取，还可以使用matplotlib或者cv2
        img = Image.open(img_file)
        
        # 将图像缩放为100*100
        # transforms.Resize(size=(100, 100)), 这种是硬拉伸【不推荐】
        # transforms.CenterCrop(size=(100, 100)), 这种是以中心为原点，直接裁剪指定的大小，如果裁大了，会给补偿黑色背景
        max_side = max(img.size)
        img = transforms.CenterCrop(size=(max_side, max_side))(img)

        # 图像数据处理
        img = self.trans(img)

        # 标签转张量
        label = torch.tensor(data=self.label2idx[img_label], dtype=torch.long)   

        return img, label

In [6]:
"""
    数据打包
"""
train_dataset = ImageDataset(root="./gestures/train/", trans=trans)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=12, shuffle=True)

test_dataset = ImageDataset(root="./gestures/test/", trans=trans)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=12, shuffle=False)

In [15]:
"""
    准备训练
"""
device = "cuda" if torch.cuda.is_available() else "cpu"
model = LeNet5().to(device=device)
optimizer = torch.optim.SGD(params=model.parameters(), lr=1e-3)
# optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
epochs = 100

In [16]:
"""
    定义过程监控函数
"""
def get_acc(data_loader):
    # 模型的 eval 模式
    model.eval()
    accs = []
    with torch.no_grad():
        for batch_X, batch_Y in data_loader:
                # 数据搬家
                batch_X = batch_X.to(device=device)
                batch_Y = batch_Y.to(device=device)
                # 正向传播
                y_pred = model(batch_X)
                # 简单解析
                y_pred = y_pred.argmax(dim=-1)
                # 计算准确率
                acc = (y_pred == batch_Y).to(dtype=torch.float32).mean().item()
                accs.append(acc)
        final_acc = torch.tensor(data=accs, dtype=torch.float32).mean().item()
    return final_acc

In [17]:
"""
    定义训练过程
"""
def train():
    # 先观察一下自然概率
    train_acc = get_acc(data_loader=train_dataloader)
    test_acc = get_acc(data_loader=test_dataloader)
    
    print(f"未训练之前，初始的状态：训练集准确率：{train_acc}, 测试集准确率：{test_acc}")
    
    for epoch in range(epochs):
        # 模型设置为训练模式
        model.train()
        for batch_idx, (batch_X, batch_Y) in enumerate(train_dataloader):
            # 数据搬家
            batch_X = batch_X.to(device=device)
            batch_Y = batch_Y.to(device=device)
            # 正向传播
            y_pred = model(batch_X)
            # 计算误差
            loss = loss_fn(y_pred, batch_Y)
            # 反向传播
            loss.backward()
            # 优化一步
            optimizer.step()
            # 清空梯度
            optimizer.zero_grad()
        # 每轮结束，做一次测试
        train_acc = get_acc(data_loader=train_dataloader)
        test_acc = get_acc(data_loader=test_dataloader)
        print(f"当前训练到第 {epoch+1} 轮，训练集准确率：{train_acc}, 测试集准确率：{test_acc}")

In [18]:
train()

未训练之前，初始的状态：训练集准确率：0.07374100387096405, 测试集准确率：0.06862745434045792
当前训练到第 1 轮，训练集准确率：0.08872901648283005, 测试集准确率：0.08578430861234665
当前训练到第 2 轮，训练集准确率：0.1025179997086525, 测试集准确率：0.11029411852359772
当前训练到第 3 轮，训练集准确率：0.11870503425598145, 测试集准确率：0.11519607901573181
当前训练到第 4 轮，训练集准确率：0.1324940174818039, 测试集准确率：0.12745098769664764


KeyboardInterrupt: 